In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob

%run "Fonctions.ipynb"

In [245]:
all_txts = find_all_extension('../Millau/Resultats/*', '.txt')

txts = []
for txt in all_txts:
    data = []
    with open(txt, 'r') as f:
        for line in f:
            data.append(line.split())
    txts.append(data)

In [138]:
pd.DataFrame(test).columns

RangeIndex(start=0, stop=12, step=1)

In [141]:
df = pd.DataFrame()
test_ = pd.DataFrame(test[1:], columns=test[0]).astype('float64')
for col in test_.columns:
    sub = test_[col].sort_values(ascending=False)
    df[col] = pd.concat([sub.head(10), sub.tail(10)], axis=0).reset_index(drop=True)

In [37]:
data = []
with open('../Millau/Resultats/2017-06-20-hh-mm-ss/2017-06-20-00-mm-ss/2017-06-20-00-03-34-150-weigh.txt', 'r') as f:
        for line in f:
            data.append(line.split())
df = pd.DataFrame(data[1:], columns=data[0]).astype('float64')

In [38]:
df.describe()

,t,A01:sig,A02:1sig',A03:1sum,A04:1diff,A05:1ax1,A06:1ax2,A07:1ax3,A08:1ax4,A09:1ax5,D10:on,D11:ax
count,2667.000000,2667.000000,2667.000000,2667.000000,2667.000000,2667.000000,2667.000000,2667.000000,2667.000000,2667.000000,2667.000000,2667.00000
mean,2.603516,0.017199,0.003118,0.005619,-0.002501,0.001318,0.001446,0.001024,0.000966,0.000865,0.357705,0.00375
std,1.503986,0.083773,0.082044,0.081539,0.007391,0.044798,0.049141,0.034814,0.032819,0.029403,0.479415,0.06113
min,0.000000,-0.192877,-0.198401,-0.185767,-0.037224,-0.073987,-0.081160,-0.057498,-0.054202,-0.048562,0.000000,0.00000
25%,1.301755,-0.009128,-0.001971,0.000000,-0.001149,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,2.603520,0.007558,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,3.905275,0.039186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000
max,5.207030,0.448694,0.441605,0.447982,0.025884,0.373433,0.409639,0.290210,0.273576,0.245107,1.000000,1.00000


In [40]:
millau = pd.read_csv('../Millau/Resultats/Fichiers Excel/conso_millau.csv', sep=';')

In [53]:
millau

,Date,Voie,Vitesse (m.s-1),Vitesse (km/h),Nombre d'essieu,Classe,Groupe essieu,Poids total,Poids Essieu 1,Poids Essieu 2,...,Unnamed: 26,Unnamed: 27,Poids total.1,Poids Essieu 1.1,Poids Essieu 2.1,Poids Essieu 3.1,Poids Essieu 4.1,Poids Essieu 5.1,Poids Essieu 6.1,Poids Essieu 7.1
0,2016-10-11-18-03-58-076,1,"23,0631","83,02716",2,40,11,16,"4,1","11,9",...,"83,02716",NaN,"156,802","40,5305","116,272",NaN,NaN,NaN,NaN,NaN
1,2016-10-11-18-04-04-365,1,"26,3918","95,01048",2,41,11,"10,6","2,2","8,4",...,"95,01048",NaN,"104,328","21,7578","82,5705",NaN,NaN,NaN,NaN,NaN
2,2016-10-11-18-08-25-205,1,"23,0631","83,02716",3,100,111,"11,5",4,"3,6",...,"83,02716",NaN,"112,448","39,4807","35,0648","37,9023",NaN,NaN,NaN,NaN
3,2016-10-11-18-10-32-003,1,"23,0631","83,02716",5,81,122,"18,2","2,4","3,4",...,"83,02716",NaN,"178,729","24,0199","33,1904","33,1904","44,1643","44,1643",NaN,NaN
4,2016-10-11-18-12-37-736,1,"22,4561","80,84196",4,58,13,"23,9","8,4","5,2",...,"80,84196",NaN,"234,196","82,0842","50,704","50,704","50,704",NaN,NaN,NaN
5,2016-10-11-18-13-34-673,1,"23,2727","83,78172",4,58,13,"9,7","3,7",2,...,"83,78172",NaN,"94,9994","35,9024","19,699","19,699","19,699",NaN,NaN,NaN
6,2016-10-11-18-17-02-841,1,"26,6667","96,00012",2,41,11,"12,9","3,2","9,7",...,"96,00012",NaN,"126,341","31,3856","94,955",NaN,NaN,NaN,NaN,NaN
7,2016-10-11-18-20-25-447,1,"23,2727","83,78172",5,113,113,"8,9","1,5","2,2",...,"83,78172",NaN,"87,387","14,4469","22,0508","16,9631","16,9631","16,9631",NaN,NaN
8,2016-10-11-18-23-00-843,1,"25,098","90,3528",5,113,113,"9,6","1,1","4,7",...,"90,3528",NaN,"94,1252","10,847","46,116","12,3874","12,3874","12,3874",NaN,NaN
9,2016-10-11-18-23-28-623,1,"23,2727","83,78172",5,110,113,30,"1,2","6,6",...,"83,78172",NaN,"294,542","11,3349","64,4055","72,9338","72,9338","72,9338",NaN,NaN
